In [21]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Import libs

In [4]:
import pandas as pd
import json
import requests
import kagglehub
import re
import os
import ast


# Import from kaggle

 https://www.kaggle.com/datasets/hugodarwood/epirecipes/data?

In [ ]:
# Download latest version
path = kagglehub.dataset_download("hugodarwood/epirecipes")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/epirecipes


In [ ]:
os.popen(f'cp {path}/full_format_recipes.json .')

In [ ]:
filename = "full_format_recipes.json"
with open(filename, 'r', encoding='utf-8') as f:
    data = json.load(f)

all_recipes = []
break_data_count = 0
data1 = pd.DataFrame(data)

recipe_data = {}
list_columns = ['directions', 'categories', 'ingredients']

for col in list_columns:
    if col in data1.columns:
        # Преобразуем списки в строки с разделителями
        data1[col] = data1[col].apply(lambda x: ' | '.join(x) if isinstance(x, list) else x)
        # if col=="ingredients"

# Конвертируем дату в datetime
data1 = data1.drop(columns=["date"], axis=1)
data1 = data1.rename(columns={'desc': 'description', "sodium":"salt"})
data1["n_ingredients"] = data1['ingredients'].str.split(' | ', regex=False).str.len().fillna(0).astype(int)

print(data1.shape)
display(data1.head())

(20130, 11)


,directions,fat,categories,calories,description,protein,rating,title,ingredients,salt,n_ingredients
0,"1. Place the stock, lentils, celery, carrot, t...",7.0,Sandwich | Bean | Fruit | Tomato | turkey | Ve...,426.0,None,30.0,2.500,"Lentil, Apple, and Turkey Wrap",4 cups low-sodium vegetable or chicken stock |...,559.0,15
1,Combine first 9 ingredients in heavy medium sa...,23.0,Food Processor | Onion | Pork | Bake | Bastill...,403.0,This uses the same ingredients found in boudin...,18.0,4.375,Boudin Blanc Terrine with Red Onion Confit,"1 1/2 cups whipping cream | 2 medium onions, c...",1439.0,28
2,In a large heavy saucepan cook diced fennel an...,7.0,Soup/Stew | Dairy | Potato | Vegetable | Fenne...,165.0,None,6.0,3.750,Potato and Fennel Soup Hodge,"1 fennel bulb (sometimes called anise), stalks...",165.0,6
3,Heat oil in heavy large skillet over medium-hi...,NaN,Fish | Olive | Tomato | Sauté | Low Fat | Low ...,NaN,The Sicilian-style tomato sauce has tons of Me...,NaN,5.000,Mahi-Mahi in Tomato Olive Sauce,2 tablespoons extra-virgin olive oil | 1 cup c...,NaN,10
4,Preheat oven to 350°F. Lightly grease 8x8x2-in...,32.0,Cheese | Dairy | Pasta | Vegetable | Side | Ba...,547.0,None,20.0,3.125,Spinach Noodle Casserole,"1 12-ounce package frozen spinach soufflé, tha...",452.0,6


## analisys

In [ ]:
data1[["calories","protein","salt","fat"]] = data1[["calories","protein","salt","fat"]].fillna(0)
data1 = data1.reindex(sorted(data1.columns), axis=1)
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20130 entries, 0 to 20129
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   calories       20130 non-null  float64
 1   categories     20111 non-null  object 
 2   description    13495 non-null  object 
 3   directions     20111 non-null  object 
 4   fat            20130 non-null  float64
 5   ingredients    20111 non-null  object 
 6   n_ingredients  20130 non-null  int64  
 7   protein        20130 non-null  float64
 8   rating         20100 non-null  float64
 9   salt           20130 non-null  float64
 10  title          20111 non-null  object 
dtypes: float64(5), int64(1), object(5)
memory usage: 1.7+ MB


# Import from kaggle

https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions?select=RAW_interactions.csv

In [ ]:
# Download latest version
path = kagglehub.dataset_download("shuyangli94/food-com-recipes-and-user-interactions")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/food-com-recipes-and-user-interactions


In [ ]:
os.popen(f'cp {path}/RAW_recipes.csv .')

In [ ]:
data2 = pd.read_csv("RAW_recipes.csv")
data2.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [ ]:
data2['nutrition'] = data2['nutrition'].apply(lambda x: ast.literal_eval(x))
data2['carbohydrate'] = data2['nutrition'].apply(lambda x: x[2])
data2['calories'] = data2['nutrition'].apply(lambda x: x[0])
data2['protein'] = data2['nutrition'].apply(lambda x: x[4])
data2['salt'] = data2['nutrition'].apply(lambda x: x[3])
data2['fat'] = data2['nutrition'].apply(lambda x: x[1])

In [ ]:
data2 = data2.drop(columns=["id",	"contributor_id",	"submitted", "n_steps", "nutrition"])
data2 = data2.rename(columns={'tags': 'categories', "steps": "directions", "name": "title"})

In [ ]:
data2['categories'] = data2['categories'].apply(lambda x: ast.literal_eval(x))
data2['ingredients'] = data2['ingredients'].apply(lambda x: ast.literal_eval(x))
data2['directions'] = data2['directions'].apply(lambda x: ast.literal_eval(x))

data2["categories"] = data2["categories"].apply(lambda x: ' | '.join(x) if isinstance(x, list) else x)
data2["ingredients"] = data2['ingredients'].apply(lambda x: ' | '.join(x) if isinstance(x, list) else x)
data2["directions"] = data2['directions'].apply(lambda x: '. '.join(x) if isinstance(x, list) else x)


## analisys


In [ ]:
# data3[["carbohydrate","calories","protein","salt","fat"]] = data3[["carbohydrate","calories","protein","salt","fat"]].fillna(0)
data2 = data2.reindex(sorted(data2.columns), axis=1)
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   calories       231637 non-null  float64
 1   carbohydrate   231637 non-null  float64
 2   categories     231637 non-null  object 
 3   description    226658 non-null  object 
 4   directions     231637 non-null  object 
 5   fat            231637 non-null  float64
 6   ingredients    231637 non-null  object 
 7   minutes        231637 non-null  int64  
 8   n_ingredients  231637 non-null  int64  
 9   protein        231637 non-null  float64
 10  salt           231637 non-null  float64
 11  title          231636 non-null  object 
dtypes: float64(5), int64(2), object(5)
memory usage: 21.2+ MB


In [ ]:
data2.head()

,calories,carbohydrate,categories,description,directions,fat,ingredients,minutes,n_ingredients,protein,salt,title
0,51.5,13.0,60-minutes-or-less | time-to-make | course | m...,autumn is my favorite time of year to cook! th...,make a choice and proceed with recipe. dependi...,0.0,winter squash | mexican seasoning | mixed spic...,55,7,2.0,0.0,arriba baked winter squash mexican style
1,173.4,0.0,30-minutes-or-less | time-to-make | course | m...,this recipe calls for the crust to be prebaked...,preheat oven to 425 degrees f. press dough int...,18.0,prepared pizza crust | sausage patty | eggs | ...,30,6,22.0,17.0,a bit different breakfast pizza
2,269.8,32.0,time-to-make | course | preparation | main-dis...,this modified version of 'mom's' chili was a h...,brown ground beef in large pot. add chopped on...,22.0,ground beef | yellow onions | diced tomatoes |...,130,13,39.0,48.0,all in the kitchen chili
3,368.1,10.0,60-minutes-or-less | time-to-make | course | m...,"this is a super easy, great tasting, make ahea...",place potatoes in a large pot of lightly salte...,17.0,spreadable cheese with garlic and herbs | new ...,45,11,14.0,2.0,alouette potatoes
4,352.9,337.0,weeknight | time-to-make | course | main-ingre...,my dh's amish mother raised him on this recipe...,"mix all ingredients& boil for 2 1 / 2 hours , ...",1.0,tomato juice | apple cider vinegar | sugar | s...,190,8,3.0,23.0,amish tomato ketchup for canning


# import from json

In [ ]:
# URL для загрузки
url = "https://raw.githubusercontent.com/mneedham/bbcgoodfood/master/stream_all.json"
filename = "stream_all.json"  # Имя выходного файла

In [ ]:
response = requests.get(url)
response.raise_for_status()  # Проверяем на ошибки HTTP

# Проверяем, что это действительно JSON
if 'application/json' not in response.headers.get('Content-Type', ''):
    print("Предупреждение: Сервер вернул не JSON-данные")

data = (response.text.replace('\"', "'")
        .replace('"{', "{")
        .replace('}"', "}")
        .replace("', ", '", ')
        .replace("': ", '": ')
        .replace(": '", ': "')
        .replace(": ['", ': ["')
        .replace(": {'", ': {"')
        .replace("'],", '"],')
        .replace("'}", '"}')
        .replace(", '", ', "')
        .replace("'page", '"page')
        .split("\n")
)
# Сохраняем в файл
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(f"Данные успешно сохранены в файл {filename}")

Предупреждение: Сервер вернул не JSON-данные
Данные успешно сохранены в файл stream_all.json


In [ ]:
filename = "stream_all.json"
with open(filename, 'r', encoding='utf-8') as f:
    data = json.load(f)

all_recipes = []
break_data_count = 0
for item in data:
    try:
        item = json.loads(item)
    except:
        break_data_count+=1
        continue
    recipe_data = {}

    # Извлекаем данные из article
    article = item['page']['article']
    recipe_data.update({
        'author': article['author'],
        'description': article['description'],
    })

    # Извлекаем данные из recipe
    recipe = item['page']['recipe']
    recipe_data.update({
        'cooking_time': recipe['cooking_time']/60,
        'prep_time': recipe['prep_time']/60,
        'categories': ' | '.join(recipe['keywords']+recipe['collections']+
         recipe['diet_types']+[recipe['cusine'],recipe['skill_level']]),
        'ingredients': ' | '.join(recipe['ingredients']),
        # 'cuisine': recipe['cusine'],
        # 'skill_level': recipe['skill_level'],
        'rating': float(recipe['ratings'])/20,
    })
    for elem in recipe['nutrition_info']:
        elem = elem.split(" ")
        if elem[0] in ['Carbohydrate','Protein','Salt','Fat']:
            name, num = elem[0].lower(), elem[1][:-1]
        elif elem[-1]=="calories":
            name, num = "calories", elem[1]
        else:
            continue
        recipe_data.update({name: float(num)})

    # Добавляем оставшиеся поля
    recipe_data['title'] = item['page']['title']

    all_recipes.append(recipe_data)

# Создаём DataFrame
data3 = pd.DataFrame(all_recipes)
data3["n_ingredients"] = data3['ingredients'].str.split(' | ', regex=False).str.len().fillna(0).astype(int)
data3['minutes'] = data3["cooking_time"]+data3["prep_time"]
data3 = data3.drop(columns=["author", "cooking_time",	"prep_time"])

print(f"Skip {break_data_count} reciept because break data")

Skip 31 reciept because break data


In [ ]:
print(data3.shape)
display(data3.head())
data3.to_csv("./stream_all.csv", sep="\t")

(11606, 12)


,description,categories,ingredients,rating,carbohydrate,calories,protein,salt,fat,title,n_ingredients,minutes
0,Keep some couscous in your cupboard as a stand...,Pesto | Tomato | Mozzarella | Couscous | Tobin...,couscous | pesto | vegetable stock | mozzarell...,4.75,32.0,363.0,13.0,1.02,22.0,Tomato &amp; mozzarella couscous salad,9,15.0
1,One of the simplest ways to showcase summer fr...,Vit C | Vitamin c | 1 of 5 a day | Baked fruit...,white wine | clear honey | nectarine | amarett...,4.80,35.0,253.0,5.0,0.10,7.0,Baked nectarines &amp; raspberries with almond...,11,45.0
2,For the ultimate indulgence look no further th...,Chocolate | Indulgent | Four-layered | Caramel...,butter | golden caster sugar | self-raising fl...,4.45,99.0,1036.0,15.0,1.25,67.0,Chocolate &amp; caramel ombre cake,23,115.0
3,This is a really easy meal to cook for friends...,Chicken breast | Chicken breasts | Chicken | B...,chicken breast | goat's cheese | thyme | tomat...,4.45,5.0,330.0,40.0,1.24,16.0,"Chicken, goat&#039;s cheese &amp; cherry tomat...",6,0.0
4,"A spicy, smoky&nbsp;chicken filling in soft to...",chicken tacos | low calorie dinner | easy taco...,skinless boneless chicken thigh | vegetable oi...,4.80,37.0,392.0,30.0,0.60,13.0,Chipotle chicken tacos with pineapple salsa,13,20.0


## analisys

In [ ]:
data3[["carbohydrate","calories","protein","salt","fat"]] = data3[["carbohydrate","calories","protein","salt","fat"]].fillna(0)
data3 = data3.reindex(sorted(data3.columns), axis=1)
data3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11606 entries, 0 to 11605
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   calories       11606 non-null  float64
 1   carbohydrate   11606 non-null  float64
 2   categories     11606 non-null  object 
 3   description    11606 non-null  object 
 4   fat            11606 non-null  float64
 5   ingredients    11606 non-null  object 
 6   minutes        11606 non-null  float64
 7   n_ingredients  11606 non-null  int64  
 8   protein        11606 non-null  float64
 9   rating         11606 non-null  float64
 10  salt           11606 non-null  float64
 11  title          11606 non-null  object 
dtypes: float64(7), int64(1), object(4)
memory usage: 1.1+ MB


# Concate datas

In [ ]:
data1_set = set(data1.columns)
data2_set = set(data2.columns)
data3_set = set(data3.columns)
sets = data1_set|data2_set|data3_set

print(data1.shape , sets-data1_set, " нет в 1")
print(data2.shape , sets-data2_set, " нет в 2")
print(data3.shape , sets-data3_set, " нет в 3")

(20130, 11) {'minutes', 'carbohydrate'}  нет в 1
(231637, 12) {'rating'}  нет в 2
(11606, 12) {'directions'}  нет в 3


In [ ]:
dfs = []
for df in [data1, data2, data3]:
    # Создаем временный DataFrame с полным набором возможных колонок
    temp_df = pd.DataFrame({
        'calories': df.get('calories'),
        'carbohydrate': df.get('carbohydrate'),
        'categories': df.get('categories'),
        'description': df.get('description'),
        'fat': df.get('fat'),
        'ingredients': df.get('ingredients'),
        'minutes': df.get('minutes'),
        'n_ingredients': df.get('n_ingredients'),
        'protein': df.get('protein'),
        'rating': df.get('rating'),
        'salt': df.get('salt'),
        'title': df.get('title'),
        'directions': df.get('directions')  # Есть только в df2 и df3
    })
    dfs.append(temp_df)

# Объединяем все таблицы
big_data = pd.concat(dfs, ignore_index=True)

<ipython-input-118-7eea038fc8b2>:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  big_data = pd.concat(dfs, ignore_index=True)


In [ ]:
# Преобразуем типы для экономии памяти
big_data['minutes'] = big_data['minutes'].fillna(0)
big_data = big_data.astype({
    'calories': 'float32',
    'carbohydrate': 'float32',
    'fat': 'float32',
    'minutes': 'int32',
    'n_ingredients': 'int16',
    'protein': 'float32',
    'rating': 'float32',
    'salt': 'float32'
})
print(big_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263373 entries, 0 to 263372
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   calories       263373 non-null  float32
 1   carbohydrate   243243 non-null  float32
 2   categories     263354 non-null  object 
 3   description    251759 non-null  object 
 4   fat            263373 non-null  float32
 5   ingredients    263354 non-null  object 
 6   minutes        263373 non-null  int32  
 7   n_ingredients  263373 non-null  int16  
 8   protein        263373 non-null  float32
 9   rating         31706 non-null   float32
 10  salt           263373 non-null  float32
 11  title          263353 non-null  object 
 12  directions     251748 non-null  object 
dtypes: float32(6), int16(1), int32(1), object(5)
memory usage: 17.6+ MB
None


### Add new column

In [ ]:
meat = ['meat', 'gammon', 'mutton', 'beef', 'pork', 'deer', 'camel', 'veal', 'lamb', 'goat', 'rabbit', 'turkey', 'steak', 'goose', 'duck', 'venison', 'elk', 'moose', 'loin', 'bacon', 'ham', 'baked ham', 'frankfurter', 'sausage', 'chicken', 'crackling']
milk = ['kaymak', 'milk', 'cream', 'butter', 'yogurt', 'ymer', 'viili', 'kefir', 'kumis', 'amasi', 'mursik', 'quark', 'acidophiline', 'matzoon', 'skyr', 'yogurt', 'qatyq', 'ryazhenka', 'varenets', 'ayran', 'doogh', 'lassi', 'leben', 'cheese', 'custard', 'casein', 'dairy products', 'milk products']
alcohol = ["'wine', 'beer', liqueur", "champagne", "ale", "gin", "cider", "brandy", "whisky", "stout", "lager", "rum", "vodka", "tequila", "absinthe", "alcohol", "ethyl", 'pastis', 'sherry', "moonshine"]
pig = ['bacon', 'ham', 'baked ham', 'frankfurter', 'sausage', "pork", "pig", 'gammon', 'crackling']
citrus = ['orange', 'grapefruit', 'lemon', 'tangerine', 'lime', 'kumquat', 'pomelo', 'yuzu', 'ugli', 'sudachi', 'pummelo', 'tangelo', 'mandarin']
fruts = ['apple', 'strawberries', 'apricot','acai','ackee','almond','amla','avocado','banana','blackberry','blackcurrant','blueberry','boysenberry','breadfruit','cactus','calabash','cantaloupe','carambola ','starfruit','carob','cashew','casimiroa','ceriman','chayote','cherimoya','cherry','chestnut','chinotto','citron','clementine','coconut','cocona','cranberry','custard','damson','date','dragonfruit','durian','elderberry','feijoa','fig','gooseberry','grape','grapefruit','guanabana (soursop)','guava','honeyberry','horned melon','huckleberry','jabuticaba','jackfruit','jambolan','jujube','juniper berry','karonda','key lime','kiwi','kumquat','langsat','lemon','lime','longan','loquat','lychee','mango','mangosteen','marionberry','medlar','melon','mulberry','nectarine','noni','orange','oroblanco','osage orange','papaya','passionfruit','pawpaw','peach','pear','persimmon','pineapple','pineberry','plum','pomegranate','pomelo','quince','raisin','rambutan','raspberry','redcurrant','rhubarb','rose apple','rowanberry','salak','saskatoon berry','sea buckthorn','sharon fruit','sloe','sour cherry','star apple','strawberry','sugar apple','surinam cherry','tamarillo','tangerine','tangelo','tangerillo','ugli fruit','vanilla bean','watermelon']
fish = ['salmon', 'trout', 'tuna', 'catfish', 'pike', 'sturgeon', 'starlet', 'dorado', 'sea bass', 'eel', 'herring', 'mackerel', 'cod', 'haddock', 'hake', 'plaice', 'flounder', 'sardine', 'perch', 'grouper', 'pike-perch', 'carp', 'bream', 'burbot', 'navaga', 'mullet', 'anchovy']
animal_root = fish + meat + milk + ["egg"]
nuts = ['coconut', "soy", 'chestnut', 'walnut', 'peanut', 'cola', 'almond', 'pistachio', 'hazelnut', 'pecan', 'brazil nut', 'pine nut', 'nutmeg', 'filbert', 'sesame', 'sunflower seeds', 'pumpkin seeds', 'water chestnut', 'macadamia nut', 'cashew', 'pignoli']

contra_dict = {
    "Halal": pig+alcohol,
    "Kashrut": [meat, milk, pig+alcohol],
    "Christian": meat+milk+alcohol+fish+["egg, bread"],
    "Vegan": animal_root,
    "Vegeterian": meat+fish,
    "Hindu": pig+["beef", "veal"],
    "Buckwheat": ["buckwheat", "wheat"],
    "Balsam of Peru": ['coffee','flavored tea','wine','beer','gin','liqueurs','apéritifs','cola','juice','citrus','marmalade','tomatoes','sauces','ketchup','chocolate','vanilla','pudding','ice cream','gum','candy'],
    "Celery": ['parsley', 'culantro', 'dill', 'coriander', 'fennel', 'cumin', 'caraway', 'celery'],
    "Egg": ["egg"],
    "Fish": fish,
    "Fruit": fruts+['tomato'],
    "citrus": citrus,
    "Garlic": ['garlic'],
    "Oats": ['oats'],
    "Corn": ["corn"],
    "Mustard": ['mustard'],
    "Milk": ['milk', "soy"],
    "Nut": nuts,
    "Poultry Meat": ['chicken', 'turkey', 'squab'],
    "Red Meat": ['beef', 'venison', 'lamb', 'pork'],
    "rice": ['rice'],
    "seafood": ['shellfish', 'mussels', 'caviar', 'shrimps', 'prawns', 'oysters', 'squids', 'crabs', 'crab', 'lobsters', 'scallops', 'octopus', 'crayfish', 'cuttlefish'],
    "Sulfites": ['zante currants', 'mulberry', 'physalis', 'aronia', 'chokeberries', 'sea-buckthorn', 'kumquats', 'white raisins', 'goji', 'cherries', 'cranberries', 'sour cherries', 'barberries', "apricots", "fig", "peache", "pear", "prune", "raisin",],
}

In [ ]:
cols = ["categories",	"ingredients", "description"]
big_data['combined'] = big_data[cols].apply(lambda row: ' _ '.join(row.values.astype(str)), axis=1)

def find_matches(text, category_dict):
    if pd.isna(text):  # Проверка на NaN
        return []
    found_categories = set()
    text_lower = str(text).lower()  # Преобразуем в строку и в нижний регистр
    for category, keywords in category_dict.items():
        if category=="Kashrut" and "Halal" not in found_categories:
            kashrut = True
            for keyword in keywords[0]:
                if re.search(r'\b' + re.escape(keyword) + r'\b', text_lower):
                    kashrut=False
                    break
            if not kashrut:
                for keyword in keywords[1]:
                    if re.search(r'\b' + re.escape(keyword) + r'\b', text_lower):
                        found_categories.add(category)
                        break
        elif "Halal" in found_categories:
            found_categories.add(category)
        else:
            for keyword in keywords:
                if re.search(r'\b' + re.escape(keyword) + r'\b', text_lower):
                    found_categories.add(category)
                    break  # Прерываем после первого совпадения в категории

    return " | ".join(list(found_categories))


big_data['contra'] = big_data['combined'].apply(lambda x: find_matches(x, contra_dict))

In [ ]:
big_data['rating'] = big_data['rating'].fillna(0)
big_data['description'] = big_data['description'].fillna('')

# Удаление полностью дублированных строк
big_data = big_data.drop_duplicates()

In [ ]:
print(big_data.shape)
big_data.head()

(261537, 15)


,calories,carbohydrate,categories,description,fat,ingredients,minutes,n_ingredients,protein,rating,salt,title,directions,combined,contra
0,426.0,NaN,Sandwich | Bean | Fruit | Tomato | turkey | Ve...,,7.0,4 cups low-sodium vegetable or chicken stock |...,0,15,30.0,2.500,559.0,"Lentil, Apple, and Turkey Wrap","1. Place the stock, lentils, celery, carrot, t...",Sandwich | Bean | Fruit | Tomato | turkey | Ve...,Balsam of Peru | Buckwheat | Poultry Meat | Ce...
1,403.0,NaN,Food Processor | Onion | Pork | Bake | Bastill...,This uses the same ingredients found in boudin...,23.0,"1 1/2 cups whipping cream | 2 medium onions, c...",0,28,18.0,4.375,1439.0,Boudin Blanc Terrine with Red Onion Confit,Combine first 9 ingredients in heavy medium sa...,Food Processor | Onion | Pork | Bake | Bastill...,Celery | Vegan | citrus | Christian | Vegeteri...
2,165.0,NaN,Soup/Stew | Dairy | Potato | Vegetable | Fenne...,,7.0,"1 fennel bulb (sometimes called anise), stalks...",0,6,6.0,3.750,165.0,Potato and Fennel Soup Hodge,In a large heavy saucepan cook diced fennel an...,Soup/Stew | Dairy | Potato | Vegetable | Fenne...,Kashrut | Poultry Meat | Celery | Vegan | Chri...
3,0.0,NaN,Fish | Olive | Tomato | Sauté | Low Fat | Low ...,The Sicilian-style tomato sauce has tons of Me...,0.0,2 tablespoons extra-virgin olive oil | 1 cup c...,0,10,0.0,5.000,0.0,Mahi-Mahi in Tomato Olive Sauce,Heat oil in heavy large skillet over medium-hi...,Fish | Olive | Tomato | Sauté | Low Fat | Low ...,Balsam of Peru | Garlic | Fruit | Vegan | Fish...
4,547.0,NaN,Cheese | Dairy | Pasta | Vegetable | Side | Ba...,,32.0,"1 12-ounce package frozen spinach soufflé, tha...",0,6,20.0,3.125,452.0,Spinach Noodle Casserole,Preheat oven to 350°F. Lightly grease 8x8x2-in...,Cheese | Dairy | Pasta | Vegetable | Side | Ba...,Egg | Vegan | Christian | Nut


In [ ]:
big_data.to_csv("/content/drive/MyDrive/Портфолио/project nlp/Big_data.csv", sep="\t")

# the biggest data

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nazmussakibrupol/3a2mext")

print("Path to dataset files:", path)

100%|██████████| 491M/491M [00:20<00:00, 25.5MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/nazmussakibrupol/3a2mext/versions/1


In [4]:
os.popen(f'cp {path}/3A2M_EXTENDED.csv .')

In [5]:
data = pd.read_csv("3A2M_EXTENDED.csv")
data.head()

,title,NER,Extended_NER,genre,label,directions
0,\t Arugula Pomegranate Salad,"[""baby spinach"", ""baby arugula"", ""pomegranate ...","['alfalfa sprouts', 'baby spinach', 'baby arug...",vegetables,4,"[""Toss together spinach and arugula, then plac..."
1,\t Black Bean And Turkey Chili,"[""olive oil"", ""yellow onion"", ""garlic"", ""groun...","['one', 'yellow onion', 'tomato paste', 'about...",sides,8,"[""Dice the onion and mince the garlic. Add the..."
2,\t Finger Lickin' Tofu Nuggets,"[""extra firm"", ""almond flour"", ""nutritional ye...","['extra firm', '2', 'coconut oil', 'almond flo...",nonveg,3,"[""Wrap the tofu in a clean tea towel and press..."
3,\t Jerk Beef Stew With Carrots And Tomatoes,"[""olive oil"", ""boneless beef chuck"", ""onion"", ...","['boneless beef chuck', '2', 'Saute', 'onion',...",vegetables,4,"[""Preheat oven to 350 degrees F."", ""Heat the o..."
4,\t Pomegranate Couscous Salad,"[""pomegranate arils"", ""whole wheat couscous"", ...","['whole wheat couscous', '10 minutes', 'lemon ...",vegetables,4,"[""Place couscous in a bowl with 11/2 cups of h..."


In [6]:
data = data.drop_duplicates()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2230998 entries, 0 to 2231142
Data columns (total 6 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   title         object
 1   NER           object
 2   Extended_NER  object
 3   genre         object
 4   label         int64 
 5   directions    object
dtypes: int64(1), object(5)
memory usage: 119.1+ MB


In [8]:
data['NER'] = data['NER'].apply(lambda x: ast.literal_eval(x))

In [9]:
data['Extended_NER'] = data['Extended_NER'].apply(lambda x: ast.literal_eval(x))

In [10]:
def clean_text(text, replacements):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s.,]', '', text)
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text.strip()

replacement_rules = {
    '"[': '"',
    '&': ' and ',
    '!': '.',
    ';': ',',
    ']"': '"',
    '", "': ' '
}

data['directions'] = data['directions'].apply(
    lambda x: clean_text(x, replacement_rules))

In [35]:
meat = ['meat', 'gammon', 'mutton', 'beef', 'pork', 'deer', 'camel', 'veal', 'lamb', 'goat', 'rabbit', 'turkey', 'steak', 'goose', 'duck', 'venison', 'elk', 'moose', 'loin', 'bacon', 'ham', 'baked ham', 'frankfurter', 'sausage', 'chicken', 'crackling']
milk = ['kaymak', 'milk', 'cream', 'butter', 'yogurt', 'ymer', 'viili', 'kefir', 'kumis', 'amasi', 'mursik', 'quark', 'acidophiline', 'matzoon', 'skyr', 'yogurt', 'qatyq', 'ryazhenka', 'varenets', 'ayran', 'doogh', 'lassi', 'leben', 'cheese', 'custard', 'casein', 'dairy products', 'milk products']
alcohol = ["'wine', 'beer', liqueur", "champagne", "ale", "gin", "cider", "brandy", "whisky", "stout", "lager", "rum", "vodka", "tequila", "absinthe", "alcohol", "ethyl", 'pastis', 'sherry', "moonshine"]
pig = ['bacon', 'ham', 'baked ham', 'frankfurter', 'sausage', "pork", "pig", 'gammon', 'crackling']
fish = ['salmon', 'trout', 'tuna', 'catfish', 'pike', 'sturgeon', 'starlet', 'dorado', 'sea bass', 'eel', 'herring', 'mackerel', 'cod', 'haddock', 'hake', 'plaice', 'flounder', 'sardine', 'perch', 'grouper', 'pike-perch', 'carp', 'bream', 'burbot', 'navaga', 'mullet', 'anchovy']

contra_dict = {
    "Christian": meat+milk+alcohol+fish+["egg, bread"],
    "Milk": milk,
    "Meat": meat,
    "Halal": pig+alcohol,
    }

In [41]:
def find_matches(list1, category_dict):
    found_category = 0
    has_common = {key: not set(list1).intersection(set(keywords)) for key, keywords in contra_dict.items()}
    if has_common["Christian"]==1:
        found_category=3
    elif has_common["Meat"]==1 and has_common["Milk"]==1 and has_common["Halal"]==1:
        found_category=2
    elif has_common["Halal"]==1:
        found_category=1
    return found_category


data['contra'] = data['NER'].apply(lambda x: find_matches(x, contra_dict))

In [42]:
data.head()

,title,NER,Extended_NER,genre,label,directions,contra
0,\t Arugula Pomegranate Salad,"[baby spinach, baby arugula, pomegranate arils...","[alfalfa sprouts, baby spinach, baby arugula, ...",vegetables,4,"Toss together spinach and arugula, then place ...",3
1,\t Black Bean And Turkey Chili,"[olive oil, yellow onion, garlic, ground turke...","[one, yellow onion, tomato paste, about 10 min...",sides,8,Dice the onion and mince the garlic. Add the o...,3
2,\t Finger Lickin' Tofu Nuggets,"[extra firm, almond flour, nutritional yeast, ...","[extra firm, 2, coconut oil, almond flour, nut...",nonveg,3,Wrap the tofu in a clean tea towel and press t...,3
3,\t Jerk Beef Stew With Carrots And Tomatoes,"[olive oil, boneless beef chuck, onion, Progre...","[boneless beef chuck, 2, Saute, onion, 1 hour ...",vegetables,4,"Preheat oven to 350 degrees F., Heat the olive...",3
4,\t Pomegranate Couscous Salad,"[pomegranate arils, whole wheat couscous, extr...","[whole wheat couscous, 10 minutes, lemon juice...",vegetables,4,Place couscous in a bowl with 112 cups of hot ...,3


In [43]:
data.to_csv("/content/drive/MyDrive/Портфолио/project nlp/Big_data.csv", sep="\t")